<a href="https://colab.research.google.com/github/rganj/aiml/blob/main/Student_MLE_MiniProject_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Build a Machine Learning Model

## Predict Total Fare on the NYC Taxi Dataset

Welcome to the NYC Taxi Fare Prediction project! In this Colab, we will continue using the NYC Taxi Dataset to predict the fare amount for taxi rides using a subset of available features. We will go through three main stages: building a baseline model, creating a full model, and performing hyperparameter tuning to enhance our predictions.

Now that you've completed exploratory data analysis on this dataset you should have a good understanding of the feature space.

## Project Objectives

The primary objectives of this project are as follows:

Baseline Model: We will start by building a simple baseline model to establish a benchmark for our predictions. This model will serve as a starting point to compare the performance of our subsequent models.

Full Model: Next, we will develop a more comprehensive model that leverages machine learning techniques to improve prediction accuracy. We will use Scikit-Learn's model pipeline to build a framework that enables rapid experimentation.

Hyperparameter Tuning: Lastly, we will optimize our full model by fine-tuning its hyperparameters. By systematically adjusting the parameters that control model behavior, we aim to achieve the best possible performance for our prediction task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, accuracy_score

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

Load the NYC taxi dataset into a Pandas DataFrame and do a few basic checks to ensure the data is loaded properly. Note, there are several months of data that can be used. For simplicity, use the Yellow Taxi 2022-01 parquet file [here](https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet). Here are your tasks:

  1. Load the `yellow_tripdata_2022-01.parquet` file into Pandas.
  2. Print the first 5 rows of data.
  3. Drop any rows of data that contain NULL values.
  4. Create a new feature, 'trip_duration' that captures the duration of the trip in minutes.
  5. Create a varible named 'target_variable' to store the name of the thing we're trying to predict, 'total_amount'.
  6. Create a list called 'feature_cols' containing the feature names that we'll be using to predict our target variable. The list should contain 'VendorID', 'trip_distance', 'payment_type', 'PULocationID', 'DOLocationID', and 'trip_duration'.

In [ ]:
# Load the dataset into a pandas DataFrame (from https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet")
df = df.sample(frac=0.1)
df.shape

(246393, 19)

In [ ]:
# Display the first few rows of the dataset
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
565745,1,2022-01-09 00:55:42,2022-01-09 01:05:02,2.0,2.30,1.0,N,170,114,1,9.5,3.0,0.5,2.65,0.00,0.3,15.95,2.5,0.0
1768806,2,2022-01-24 00:38:14,2022-01-24 00:51:01,1.0,4.04,1.0,N,162,112,1,14.5,0.5,0.5,3.66,0.00,0.3,21.96,2.5,0.0
1029838,2,2022-01-15 00:43:54,2022-01-15 00:49:46,1.0,1.22,1.0,N,79,234,2,6.0,0.5,0.5,0.00,0.00,0.3,9.80,2.5,0.0
189866,1,2022-01-04 00:18:43,2022-01-04 00:58:40,1.0,16.00,1.0,N,48,7,2,48.5,3.0,0.5,0.00,6.55,0.3,58.85,2.5,0.0
1675292,2,2022-01-22 20:43:40,2022-01-22 21:04:16,2.0,2.84,1.0,N,170,249,1,14.5,0.5,0.5,3.66,0.00,0.3,21.96,2.5,0.0


In [ ]:
# Drop rows with missing values.
df.dropna(axis=0,inplace=True)
df.isnull().sum()

,0
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
RatecodeID,0
store_and_fwd_flag,0
PULocationID,0
DOLocationID,0
payment_type,0


In [ ]:
# Create new feature, 'trip_duration'.
df['trip_duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [ ]:
# Create a list called feature_col to store column names
feature_col = ['trip_distance', 'trip_duration', 'PULocationID', 'DOLocationID', 'payment_type']
print(feature_col)

['trip_distance', 'trip_duration', 'PULocationID', 'DOLocationID', 'payment_type']


Splitting a dataset into training and test sets is a crucial step in machine learning model development. It allows us to evaluate the performance and generalization ability of our models accurately. The training set is used to train the model, while the test set serves as an independent sample for evaluating its performance.

1. **Model Training**: The training set is used to fit the model, allowing it to learn the underlying patterns and relationships between the features and the target variable. By exposing the model to a diverse range of examples in the training set, it can capture the underlying structure of the data.

2. **Model Evaluation**: The test set, which is independent of the training set, is crucial for evaluating how well the trained model generalizes to unseen data. It provides an unbiased assessment of the model's performance on new instances. By measuring the model's accuracy, precision, recall, or other evaluation metrics on the test set, we can estimate how well the model will perform on unseen data.

3. **Preventing Overfitting**: Overfitting occurs when a model learns the training data's noise and idiosyncrasies instead of the underlying patterns. By evaluating the model on the test set, we can identify if the model is overfitting. If the model performs significantly worse on the test set compared to the training set, it indicates overfitting. In such cases, we might need to adjust the model, feature selection, or regularization techniques to improve generalization.

4. **Hyperparameter Tuning**: Splitting the dataset allows us to perform hyperparameter tuning on the model. Hyperparameters are configuration settings that control the learning process, such as learning rate, regularization strength, or the number of hidden layers in a neural network. By using a validation set (often created from a portion of the training set), we can iteratively adjust the hyperparameters and select the best combination that maximizes the model's performance on the validation set. The final evaluation on the test set provides an unbiased estimate of the model's performance.

By splitting the dataset into training and test sets, we can ensure that our models are both well-trained and accurately evaluated. This separation helps us understand how the model will perform on new, unseen data, which is critical for assessing its effectiveness and making informed decisions about its deployment.

Here is your task:

  1. Use Scikit-Learn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and test sets. Don't forget to set the random state.

In [ ]:
# Split dataset into training and test sets
target_variable = df['total_amount']
X = df[feature_col]  # Features
y = target_variable  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

The importance of a baseline model, even if it uses a simple strategy like always predicting the mean, cannot be understated. Here's why a baseline model is valuable:

1. **Performance Comparison**: A baseline model serves as a reference point for evaluating the performance of more sophisticated models. By establishing a simple yet reasonable baseline, we can determine whether our advanced models offer any significant improvement over this basic approach. It helps us set realistic expectations and gauge the effectiveness of our efforts.

2. **Model Complexity**: A baseline model provides insight into the complexity required to solve the prediction task. If a simple strategy like predicting the median performs reasonably well, it suggests that the problem might not necessitate complex modeling techniques. Conversely, if the baseline model performs poorly, it indicates the presence of more intricate patterns that need to be captured by more sophisticated models.

3. **Minimum Performance Requirement**: A baseline model can establish a minimum performance requirement for a predictive task. If we cannot outperform the baseline, it suggests that our models have failed to capture even the most fundamental relationships within the data. In such cases, we may need to revisit our data preprocessing steps, feature engineering techniques, or consider other external factors affecting the task.

4. **Identifying Data Issues**: A baseline model can help identify potential issues within the dataset. If the baseline model performs poorly, it may indicate problems like missing values, outliers, or data inconsistencies. These issues can be further investigated and resolved to improve the overall model performance.

While a baseline model like always predicting the median may not offer the highest prediction accuracy, its importance lies in its role as a starting point for model development and evaluation. It provides a solid foundation for comparing and assessing the performance of more complex models, ensuring that any improvements made are meaningful and significant.

Here is your task:

  1. Create a model that always predicts the mean total fare of the training dataset. Use Scikit-Learn's [mean_absolute_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html) to evaluate this model. Is it any good?

With a baseline metric in place, we can try to build a machine learning model. Obviously, if the model can't beat the baseline then there are some major issues to be resolved.

It's always a good idea to start with a simple machine learning model, like linear regression, and build upon it if necessary.

Here are your tasks:

  1. Use Scikit-Learn's [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to preprocess the categorical and continuous features independently. Apply the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to the continuous columns and [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to the categorical columns.

  One-hot encoding is a popular technique used to represent categorical variables numerically in machine learning models. It transforms categorical features into a binary vector representation, where each category is represented by a binary column. Here's an explanation of one-hot encoding:

  When working with categorical variables, such as colors (e.g., red, blue, green) or vehicle types (e.g., car, truck, motorcycle), machine learning algorithms often require numerical inputs. However, directly assigning numerical values to categories can introduce unintended relationships or orderings between them. For example, assigning the values 0, 1, and 2 to the categories red, blue, and green may imply a sequential relationship, which is not desired.

  One-hot encoding solves this problem by creating new binary columns, equal to the number of unique categories in the original feature. Each binary column represents a specific category and takes a value of 1 if the data point belongs to that category, and 0 otherwise. This encoding ensures that no implicit ordering or relationship exists between the categories.

  2. Integrate the preprocessor in the previous step with Scikit-Learn's [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) model using a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

  3. Train the pipeline on the training data.

  4. Evaluate the model using mean absolute error as a metric on the test data. Does the model beat the baseline?


In [ ]:
# Use Scikit-Learn's ColumnTransformer to preprocess the categorical and
# continuous features independently.

categorical_transform = OneHotEncoder()
continuous_transform = StandardScaler()

categorical_features = ['payment_type']
continuous_features = ['trip_distance', 'trip_duration', 'PULocationID', 'DOLocationID']
transformer = ColumnTransformer(transformers=[('continuous', continuous_transform, continuous_features), ('categorical', categorical_transform, categorical_features)])

#model = LinearRegression()

#pipeline = Pipeline([('preprocessor', transformer), ('model', model)])

#pipeline.fit(X_train, y_train)

#y_pred = pipeline.predict(X_test)
#l = mean_absolute_error(y_test, y_pred)
#print("MAE of Linear Regression on training set: {:.3f}".format(l))

In [ ]:
# Create a pipeline object containing the column transformations and regression
# model.

model = LinearRegression()
pipeline = Pipeline([('preprocessor', transformer), ('model', model)])

In [ ]:
# Fit the pipeline on the training data.

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  StandardScaler(),
                                                  ['trip_distance',
                                                   'trip_duration',
                                                   'PULocationID',
                                                   'DOLocationID']),
                                                 ('categorical',
                                                  OneHotEncoder(),
                                                  ['payment_type'])])),
                ('model', LinearRegression())])

In [ ]:
# Make predictions on the test data.

y_pred = pipeline.predict(X_test)

l = mean_absolute_error(y_test, y_pred)
print("MAE of Linear Regression on training set: {:.3f}".format(l))

MAE of Linear Regression on training set: 2.764


Random Forest Regression and Linear Regression are two commonly used regression algorithms, each with its own advantages and suitability for different scenarios. Random Forest Regression offers several advantages over Linear Regression, including:

1. **Non-linearity**: Random Forest Regressor is capable of capturing non-linear relationships between features and the target variable. In contrast, Linear Regression assumes a linear relationship between the features and the target. When faced with non-linear relationships or complex feature interactions, Random Forest Regressor can provide more accurate predictions.

2. **Robustness to Outliers**: Random Forest Regressor is generally more robust to outliers compared to Linear Regression. Outliers can disproportionately impact the coefficients and predictions of Linear Regression models. However, as an ensemble of decision trees, Random Forest Regressor can mitigate the effect of outliers by averaging predictions from multiple trees.

3. **Feature Importance**: Random Forest Regressor provides a measure of feature importance, which helps identify the most influential features for making predictions. This information is useful for feature selection, understanding the underlying relationships in the data, and gaining insights into the problem domain. Unlike Linear Regression, which provides coefficient values indicating the direction and magnitude of relationships, Random Forest Regressor explicitly highlights feature importance.

4. **Handling of Categorical Variables**: Random Forest Regressor can effectively handle categorical variables without requiring pre-processing steps like one-hot encoding. It can directly incorporate categorical variables into the model, making it more convenient when working with mixed data types. In contrast, Linear Regression often requires categorical variables to be encoded or transformed before use.

5. **Handling of High-Dimensional Data**: Random Forest Regressor can handle datasets with a large number of features (high dimensionality) by automatically selecting subsets of features during the construction of individual decision trees. This reduces the risk of overfitting, which is a concern with Linear Regression when dealing with high-dimensional data.

6. **Resistance to Multicollinearity**: Random Forest Regressor is less affected by multicollinearity, which occurs when predictor variables are highly correlated. In Linear Regression, highly correlated features can lead to unstable coefficient estimates, making it challenging to interpret the individual effects of each feature. Random Forest Regressor, as an ensemble approach, is less impacted by multicollinearity because each tree is built independently.

Here are your tasks:

  1. Build a Random Forest Regressor model using Scikit-Learn's [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) and train it on the train data.

  2. Evaluate the performance of the model on the test data using mean absolute error as a metric. Mess around with various input parameter configurations to see how they affect the model. Can you beat the performance of the linear regression model?

In [ ]:
# Build random forest regressor model

model = RandomForestRegressor(random_state = 100,bootstrap = True, max_depth=1,max_features=2,min_samples_leaf=3,min_samples_split=5,n_estimators=3)


In [ ]:
# Make predictions on the test data

pipeline = Pipeline([('preprocessor', transformer), ('model', model)])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
l = mean_absolute_error(y_test, y_pred)
print("MAE of Random Forest Regressor on training set: {:.3f}".format(l))


MAE of Random Forest Regressor on training set: 7.228


Hyperparameter tuning plays a critical role in machine learning model development. It involves selecting the optimal values for the hyperparameters, which are configuration settings that control the behavior of the learning algorithm. Here's why hyperparameter tuning is so important in ML:

1. **Optimizing Model Performance**: The choice of hyperparameters can significantly impact the model's performance. By fine-tuning the hyperparameters, we can improve the model's accuracy, precision, recall, or other performance metrics. It helps to extract the maximum predictive power from the chosen algorithm and ensures that the model is well-suited to the specific problem at hand.

2. **Avoiding Overfitting and Underfitting**: Hyperparameter tuning helps strike a balance between overfitting and underfitting.

3. **Exploring Model Complexity**: Hyperparameter tuning enables us to explore the complexity of the model. For instance, in algorithms like decision trees or neural networks, we can adjust the number of layers, the number of neurons, or the maximum depth of the tree. By systematically modifying these hyperparameters, we can understand how different levels of complexity impact the model's performance and find the right balance between simplicity and complexity.

Note, there are multiple approaches to hyperparemeter tuning.  

While grid search is the easiest to understand and implement there are many advantages of Bayesian search over grid search for hyperparameter tuning:

1. **Efficiency**: Bayesian search is generally more efficient than grid search. Grid search explores all possible combinations of hyperparameter values, which can be computationally expensive and time-consuming, especially when dealing with a large number of hyperparameters or a wide range of values. Bayesian search, on the other hand, intelligently selects the next hyperparameter configuration to evaluate based on the results of previous evaluations. It focuses on areas of the hyperparameter space that are more likely to yield better performance, reducing the number of evaluations needed.

2. **Flexibility**: Bayesian search is flexible in handling continuous and discrete hyperparameters. It can handle both types of hyperparameters naturally and effectively. In contrast, grid search is more suitable for discrete hyperparameters but may struggle with continuous ones, as it requires discretization or defining a finite set of values to search over.

3. **Adaptive Search**: Bayesian search adapts its search strategy based on the results of previous evaluations. It maintains a probability distribution over the hyperparameter space, updating it with each evaluation. This allows it to dynamically allocate more evaluations to promising regions and explore unexplored areas. In contrast, grid search follows a fixed and predefined search grid, regardless of the results of previous evaluations.

4. **Better Convergence**: Bayesian search has the potential to converge to the optimal hyperparameter configuration more quickly.

Here are your tasks:

  1. Perform a grid-search on a Random Forest Regressor model. Only search the space for the parameters 'n_estimators', 'max_depth', and 'min_samples_split'. Note, this can take some time to run. Make sure you set reasonable boundaries for the search space. Use Scikit-Learn's [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) method.

  2. After you've identified the best parameters, train a random forest regression model using these parameters on the full training data.

  3. Evaluate the model from the previous step using the test data. How does your model perform?

In [ ]:
# Define the hyperparameters to tune.
param_grid = [{
    'max_depth': [80, 90, 110],
    'min_samples_split': [2, 10, 12],
    'n_estimators': [100, 200, 300]
}]

In [ ]:
# Perform grid search to find the best hyperparameters. This could take a while.
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, cv = 2, verbose = 2)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
best_model = grid_search.best_estimator_
print(best_model)

Fitting 2 folds for each of 27 candidates, totalling 54 fits
[CV] END max_depth=80, min_samples_split=2, n_estimators=100; total time=  16.6s
[CV] END max_depth=80, min_samples_split=2, n_estimators=100; total time=  16.9s
[CV] END max_depth=80, min_samples_split=2, n_estimators=200; total time=  34.1s
[CV] END max_depth=80, min_samples_split=2, n_estimators=200; total time=  33.4s
[CV] END max_depth=80, min_samples_split=2, n_estimators=300; total time=  50.0s
[CV] END max_depth=80, min_samples_split=2, n_estimators=300; total time=  49.9s
[CV] END max_depth=80, min_samples_split=10, n_estimators=100; total time=  16.5s
[CV] END max_depth=80, min_samples_split=10, n_estimators=100; total time=  15.6s
[CV] END max_depth=80, min_samples_split=10, n_estimators=200; total time=  31.7s
[CV] END max_depth=80, min_samples_split=10, n_estimators=200; total time=  31.5s
[CV] END max_depth=80, min_samples_split=10, n_estimators=300; total time=  47.4s
[CV] END max_depth=80, min_samples_split=10

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


{'max_depth': 80, 'min_samples_split': 10, 'n_estimators': 300}
0.8723559593599293
RandomForestRegressor(max_depth=80, max_features=2, min_samples_leaf=3,
                      min_samples_split=10, n_estimators=300, random_state=100)


In [ ]:
# Get the best model and its parameters.
random_grid = [{
    'max_depth': [80, 90, 110],
    'min_samples_split': [2, 10, 12],
    'n_estimators': [100, 200, 300]
}]

random_search_cv = RandomizedSearchCV(model, random_grid, cv=3, n_iter=2, n_jobs = -1, verbose = 2)
random_search_cv.fit(X_train, y_train)
print(random_search_cv.best_params_)
print(random_search_cv.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
{'n_estimators': 300, 'min_samples_split': 2, 'max_depth': 110}
0.8746214918766095


In [ ]:
# Fit the best classifier on the training data.
best_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=80, max_features=2, min_samples_leaf=3,
                      min_samples_split=10, n_estimators=300, random_state=100)

In [ ]:
# Make predictions on the test data
y_pred = best_model.predict(X_test)
l = mean_absolute_error(y_test, y_pred)
print("MAE of Random Forest Regressor on training set: {:.3f}".format(l))

MAE of Random Forest Regressor on training set: 1.558
